In [1]:
from transformers import (pipeline, BertTokenizer, BertForMaskedLM, BertConfig, DataCollatorForLanguageModeling,
                            Trainer, TrainingArguments)
import torch
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets


In [2]:

unmasker = pipeline('fill-mask', model='bert-base-uncased')

unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731099545955658,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i'm a fashion model."},
 {'score': 0.08774477988481522,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.05338403955101967,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.04667219892144203,
  'token': 3565,
  'token_str': 'super',
  'sequence': "hello i'm a super model."},
 {'score': 0.02709587849676609,
  'token': 2986,
  'token_str': 'fine',
  'sequence': "hello i'm a fine model."}]

In [19]:
config = BertConfig.from_pretrained('bert-base-uncased')
model_path = "best_model.pt"
state_dict = torch.load(model_path)
model = BertForMaskedLM(config)
model.load_state_dict(state_dict)
model.eval()

RuntimeError: Error(s) in loading state_dict for BertForMaskedLM:
	Missing key(s) in state_dict: "cls.predictions.bias", "cls.predictions.transform.dense.weight", "cls.predictions.transform.dense.bias", "cls.predictions.transform.LayerNorm.weight", "cls.predictions.transform.LayerNorm.bias", "cls.predictions.decoder.weight", "cls.predictions.decoder.bias". 
	Unexpected key(s) in state_dict: "classifier.weight", "classifier.bias", "bert.pooler.dense.weight", "bert.pooler.dense.bias". 

In [ ]:
text = "The quick brown [MASK] jumps over the lazy dog."
tokenized_text = text.split()
tokenized_text[3] = '[MASK]'
masked_text = ' '.join(tokenized_text)

input = tokenizer.encode(masked_text, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:

    print(tokenizer.decode(token))




c a t
d o g
b e a r
m a n
# # i e


In [20]:
from transformers import BertTokenizer, BertForMaskedLM
from sklearn.metrics import accuracy_score
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the model's predictions
predicted_words = []

for sentence in sentences:
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Use the model to predict the masked token
    outputs = model(**inputs)
    predictions = outputs.logits[0, mask_index, :]

    # Get the predicted token
    predicted_token = torch.argmax(predictions)
    predicted_word = tokenizer.decode([predicted_token])

    # Add the predicted word to the list
    predicted_words.append(predicted_word)

# Calculate the accuracy of the model's predictions
accuracy = accuracy_score(actual_words, predicted_words)

print(f"Accuracy: {accuracy}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 0.0


In [15]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the losses
losses = []

for sentence, actual_word in zip(sentences, actual_words):
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Replace the masked token with the actual token
    inputs["input_ids"][0, mask_index] = tokenizer.encode(actual_word, add_special_tokens=False)[0]

    # Use the model to calculate the loss
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

    # Add the loss to the list
    losses.append(loss.item())

# Calculate the average loss
average_loss = sum(losses) / len(losses)

print(f"Average Loss: {average_loss}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Loss: 3.7346343994140625


In [16]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [17]:
wiki_dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [6]:
wiki_dataset["train"][4]["text"]

" The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game 's opening theme was sung by May 'n . \n"

In [18]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [19]:
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


In [7]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

class TrainerWithCustomLoss(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        print("logits: ", logits)
        print("labels: ", labels)
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))
        print("loss: ", loss)
        return (loss, outputs) if return_outputs else loss
    """
    def compute_loss(self, model, inputs, return_outputs=False):
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)

        # code for calculating accuracy
        if "labels" in inputs:
            logits = outputs.logits.detach()
            preds = logits.argmax(dim=1)
            acc1 = metric.compute(predictions=preds, references=inputs.labels)
            self.log({'accuracy_score': acc1})
            acc = (
                (preds.argmax(axis=-1) == inputs.labels.reshape(1, len(inputs.labels))[0])
                .type(torch.float)
                .mean()
                .item()
            )
            self.log({"train_accuracy": acc})
        # end code for calculating accuracy
                    
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss
    """


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator
)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

logits:  tensor([[[ -6.7583,  -6.6465,  -6.6387,  ...,  -5.9257,  -6.1770,  -3.5506],
         [-12.9563, -12.5379, -12.6684,  ..., -10.0992,  -9.8837, -10.7693],
         [ -6.3406,  -6.2203,  -6.3291,  ...,  -6.7993,  -7.6033,  -2.1346],
         ...,
         [ -6.5742,  -6.5179,  -6.5060,  ...,  -6.9608,  -7.8442,  -2.5688],
         [ -6.5262,  -6.4379,  -6.4530,  ...,  -6.8185,  -7.8497,  -2.3651],
         [ -5.9338,  -5.8018,  -5.8560,  ...,  -6.1339,  -7.2824,  -2.4826]],

        [[ -6.8234,  -6.7755,  -6.7902,  ...,  -6.1639,  -5.9962,  -3.9075],
         [ -7.7885,  -7.7508,  -7.5184,  ...,  -7.8819,  -8.0897,  -2.6411],
         [ -8.9176,  -9.2189,  -9.0491,  ...,  -9.6174,  -8.6530,  -6.2630],
         ...,
         [ -5.6950,  -5.7152,  -5.6204,  ...,  -5.5679,  -5.2874,   0.4978],
         [ -5.4861,  -5.4906,  -5.4576,  ...,  -5.5444,  -5.3508,   0.6037],
         [ -5.5245,  -5.5295,  -5.4850,  ...,  -5.5602,  -5.3779,   0.5525]],

        [[ -6.7583,  -6.6465,  -6.6

logits:  tensor([[[ -6.7461,  -6.7388,  -6.7324,  ...,  -6.1944,  -5.8868,  -4.2601],
         [ -8.2240,  -8.0177,  -8.3749,  ...,  -7.1757,  -7.2678,  -6.3609],
         [ -5.8477,  -5.6228,  -5.7920,  ...,  -5.1521,  -4.9341,  -3.3612],
         ...,
         [ -8.5617,  -8.3995,  -8.6617,  ...,  -8.8486,  -6.5635,  -4.4154],
         [-13.9054, -13.9440, -14.4776,  ..., -13.6310, -10.0956,  -9.3304],
         [ -5.7680,  -5.8445,  -5.8600,  ...,  -4.9976,  -5.6978,  -5.9159]],

        [[ -6.9659,  -6.9073,  -6.8668,  ...,  -6.4061,  -6.2081,  -4.4244],
         [-10.1949, -10.2289, -10.1305,  ...,  -9.0903,  -8.7840,  -7.7560],
         [ -1.2380,  -1.2418,  -1.1435,  ...,  -1.9111,  -2.0167,  -6.1806],
         ...,
         [ -8.5828,  -8.2562,  -8.4341,  ...,  -7.3994,  -6.7373,  -8.5761],
         [ -6.6417,  -6.4679,  -6.7177,  ...,  -6.3766,  -6.5059,  -3.4220],
         [-11.9629, -12.0953, -12.0355,  ..., -11.0534,  -8.6478,  -8.8394]],

        [[ -6.7583,  -6.6465,  -6.6

KeyboardInterrupt: 

In [23]:
tokenizer = BertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = BertForMaskedLM.from_pretrained('distilbert/distilbert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', '

In [24]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

{'eval_loss': 10.404870986938477,
 'eval_runtime': 210.8167,
 'eval_samples_per_second': 20.672,
 'eval_steps_per_second': 1.295}

# Try to make BERT degenerate

In [8]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


# training with high lr
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=1
)


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    train_dataset=wiki_dataset_tokenized["train"].select(range(100)),
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator

)

# get loss on test set
trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


logits:  tensor([[[ -6.6992,  -6.6638,  -6.6924,  ...,  -6.0393,  -5.9508,  -3.5748],
         [ -5.9749,  -6.0426,  -6.1014,  ...,  -5.9524,  -7.0385,  -3.9011],
         [-10.8663, -11.0067, -11.2068,  ...,  -7.8600, -10.4395,  -7.8162],
         ...,
         [ -7.8557,  -8.0231,  -7.6362,  ...,  -6.1552,  -6.0496,  -4.8558],
         [ -6.1210,  -6.4641,  -6.1488,  ...,  -4.7911,  -5.8879,  -5.2471],
         [ -7.2318,  -7.1948,  -7.0307,  ...,  -6.9691,  -8.0581,  -3.3280]],

        [[ -6.7442,  -6.7165,  -6.7381,  ...,  -6.0821,  -5.9402,  -3.9106],
         [ -8.5159,  -8.9991,  -8.6121,  ...,  -8.3131,  -8.7912,  -9.5860],
         [ -7.2958,  -7.4657,  -7.1546,  ...,  -6.1452,  -7.1026,  -2.2832],
         ...,
         [ -8.0035,  -7.9842,  -7.9967,  ...,  -7.5601,  -7.4848,  -6.8501],
         [ -7.1085,  -7.2498,  -7.2445,  ...,  -6.6762,  -7.0324,  -4.6577],
         [ -7.6700,  -7.7727,  -7.7464,  ...,  -7.5336,  -6.9656,  -7.0016]],

        [[ -7.0935,  -7.1205,  -7.0

Step,Training Loss


logits:  tensor([[[ -6.7764,  -6.7404,  -6.7039,  ...,  -6.1530,  -5.9560,  -3.4166],
         [ -8.6935,  -8.6494,  -8.2257,  ...,  -7.4427,  -8.4708,  -5.4229],
         [ -1.7885,  -1.3541,  -1.4196,  ...,  -0.6504,  -1.7922,  -1.0218],
         ...,
         [ -7.5721,  -7.5665,  -7.4560,  ...,  -6.5662,  -7.2993,  -1.3928],
         [ -9.8270,  -9.9423, -10.1175,  ...,  -8.7583, -10.2672,  -4.0271],
         [-16.8637, -17.1110, -16.7067,  ..., -14.6127, -14.9300, -10.2000]],

        [[ -6.9787,  -6.9599,  -6.9546,  ...,  -6.1410,  -6.1578,  -3.8571],
         [ -6.7116,  -6.5687,  -6.4282,  ...,  -6.6284,  -6.7072,  -3.8482],
         [ -5.4049,  -5.3169,  -5.1329,  ...,  -4.7682,  -5.7480,  -1.9745],
         ...,
         [ -6.0889,  -6.1732,  -6.0806,  ...,  -5.9857,  -7.0053,  -4.5634],
         [ -6.6698,  -6.5696,  -6.4529,  ...,  -6.3663,  -6.6647,  -2.6759],
         [ -6.0977,  -6.1820,  -6.1096,  ...,  -6.2007,  -6.5402,  -2.6801]],

        [[ -7.2722,  -7.1827,  -7.2

logits:  tensor([[[ -6.9065,  -6.9297,  -6.8932,  ...,  -6.2447,  -6.2182,  -4.2884],
         [-11.0507, -10.9604, -11.3159,  ...,  -9.7047,  -9.6166,  -8.4840],
         [ -8.4413,  -8.9349,  -8.4520,  ...,  -7.1246,  -8.2517,  -6.3788],
         ...,
         [ -6.6182,  -6.6686,  -6.8117,  ...,  -6.5088,  -8.0175,  -6.3456],
         [ -6.1215,  -6.3211,  -6.5225,  ...,  -6.2588,  -6.6689,  -8.4237],
         [ -6.0221,  -6.2126,  -6.3862,  ...,  -6.3516,  -6.5770,  -8.4318]],

        [[ -6.7583,  -6.6466,  -6.6387,  ...,  -5.9257,  -6.1770,  -3.5506],
         [-12.9563, -12.5379, -12.6684,  ..., -10.0992,  -9.8837, -10.7693],
         [ -6.3406,  -6.2203,  -6.3291,  ...,  -6.7993,  -7.6033,  -2.1346],
         ...,
         [ -6.5742,  -6.5179,  -6.5060,  ...,  -6.9608,  -7.8442,  -2.5688],
         [ -6.5262,  -6.4379,  -6.4530,  ...,  -6.8185,  -7.8497,  -2.3651],
         [ -5.9338,  -5.8018,  -5.8560,  ...,  -6.1339,  -7.2824,  -2.4826]],

        [[ -6.8228,  -6.7997,  -6.7

KeyboardInterrupt: 

In [ ]:

unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

unmasker("Hello I'm a [MASK] model.")

[{'score': 0.10727187246084213,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.05541478469967842,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i'm a fashion model."},
 {'score': 0.04304599016904831,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.036487236618995667,
  'token': 3565,
  'token_str': 'super',
  'sequence': "hello i'm a super model."},
 {'score': 0.0361301526427269,
  'token': 2944,
  'token_str': 'model',
  'sequence': "hello i'm a model model."}]